# Montecarlo simulation of the Ising model 

In [2]:
import numpy as np
from collections import defaultdict

defaultdict は Python の collections モジュールにあるクラスで、通常の辞書 (dict) を拡張したものです。defaultdict を使うと、辞書に存在しないキーにアクセスした場合に自動的に初期値を生成する機能が加わります。

In [3]:
from collections import defaultdict

# 初期値を0に設定するdefaultdict
dd = defaultdict(int)

# 存在しないキーにアクセスすると自動的に初期値が設定される
print(dd["a"])  # 出力: 0
print(dd["b"])  # 出力: 0

# 値を設定する
dd["a"] += 1
print(dd["a"])  # 出力: 1

# 辞書の内容を確認
print(dd)  # 出力: defaultdict(<class 'int'>, {'a': 1, 'b': 0})


0
0
1
defaultdict(<class 'int'>, {'a': 1, 'b': 0})


In [11]:
dd = defaultdict(list)
print(dd["a"])
dd["a"].append(1)
dd["a"].append(2)
print(dd["a"])

[]
[1, 2]


In [17]:
class MonteCarloFirstVisit:
    def __init__(self, states, gamma=0.9):
        self.V = {s: 0.0 for s in states}  # 状態価値関数の初期化
        self.returns = defaultdict(list)  # 状態ごとの収益リスト
        self.gamma = gamma                # 割引率

    def generate_episode(self, policy, max_steps=100):
        """
        方策に従ってエピソードを生成する関数。
        エピソード = [(state, action, reward), ...]
        """
        episode = []
        state = np.random.choice(list(self.V.keys()))  # ランダムに初期状態を選択
        for _ in range(max_steps):
            action = policy(state)  # 方策に基づいて行動を選択
            next_state = np.random.choice(list(self.V.keys()))  # 仮の遷移
            reward = np.random.normal(0, 1)  # 仮の報酬（ランダムに設定）
            episode.append((state, action, reward))
            state = next_state
            if np.random.rand() < 0.1:  # 10%の確率でエピソードを終了
                break
        return episode
    
    def update(self,episode):
        """
        エピソードを基に状態価値関数を更新する関数。
        """
        visited_states = set()
        for t, (state, action, reward) in enumerate(episode):
            if state not in visited_states:
                G = 0
                for s,a,r in reversed(episode[t:]):
                    G = self.gamma*G + r
               
                self.returns[state].append(G)
                self.V[state] = np.mean(self.returns[state])
                visited_states.add(state)



In [18]:
def random_policy(state):
    return np.random.choice(["action1", "action2", "action3"])

In [19]:
states = ["A", "B", "C", "D", "E"]  # 仮の状態集合
mc = MonteCarloFirstVisit(states)

# 複数のエピソードを生成して状態価値関数を更新
for i in range(1000):  # 1000エピソード
    episode = mc.generate_episode(random_policy)
    mc.update(episode)

# 結果を出力
print("推定された状態価値関数:")
for state, value in mc.V.items():
    print(f"状態 {state}: {value:.2f}")

推定された状態価値関数:
状態 A: -0.02
状態 B: -0.05
状態 C: -0.02
状態 D: -0.11
状態 E: -0.00
